### Importing libraries

In [1]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/dual_arm_cartesian/'

In [2]:
def get_random_joints(low_limit, up_limit):
    dof = len(low_limit)
    joint_angle = np.random.rand(dof)*(up_limit-low_limit) + low_limit     
    return joint_angle


### Setting the trajopt

In [3]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.Environment()
env.StopSimulation()
env.Load("robots/pr2-beta-static.zae")
env.Load("../env/bookshelves.env.xml")
env.SetDefaultViewer()

True

### Setting the robot

In [4]:
robot = env.GetRobots()[0]

left_arm = robot.GetManipulator("leftarm")
right_arm = robot.GetManipulator("rightarm")

robot.SetActiveDOFs(np.r_[right_arm.GetArmIndices(), left_arm.GetArmIndices()])
coeffs = [1,1,1,1,1,1,1,  1,1,1,1,1,1,1,] #coefficients for joint velocity cost

dof = len(robot.GetActiveDOFValues())
left_dof_away = np.array([ 0.5646,  1.1371, -0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
right_dof_away = np.array([ -0.5646,  1.1371, 0.65  , -2.1172,  2.0552, 0.99, -2.168 ])
robot.SetDOFValues(left_dof_away, left_arm.GetArmIndices())
robot.SetDOFValues(right_dof_away, right_arm.GetArmIndices())
init_joint0 = robot.GetActiveDOFValues()

low_limit = robot.GetActiveDOFLimits()[0]
up_limit = robot.GetActiveDOFLimits()[1]
low_limit[4] = -np.pi
low_limit[6] = -np.pi
up_limit[4] = np.pi
up_limit[6] = np.pi


### Set the IK Solver 

In [5]:
hmat_target = np.array([[ 0.9689,  0.    , -0.2474,  0.4918],
       [-0.    ,  1.    ,  0.    ,  0.912 ],
       [ 0.2474, -0.    ,  0.9689,  0.8947],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])
quat_target = openravepy.quatFromRotationMatrix(hmat_target).tolist()
xyz_target = hmat_target[0:3,3].tolist()

# BEGIN IK for all manips
manips = [left_arm,right_arm]
for manip in manips:
    robot.SetActiveManipulator(manip)
    ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(
        robot, iktype=openravepy.IkParameterization.Type.Transform6D)
    if not ikmodel.load():
        ikmodel.autogenerate()   
    
target_joint = ku.ik_for_link(hmat_target, left_arm, "l_gripper_tool_frame",
    filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
# END ik

### Construct a random problem 

### Build / Retrieve Dataset

In [6]:
def build_database(x_init, y_init, num_traj, limits, n_steps = 30,predictor=None):
    num_plan = 0
    comp_times = []
    costs = []
    tic = time.time()
    
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        robot.SetActiveDOFValues(init_joint0)
        target_joint_right,xyz_target_right,target_joint_left,xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        
        quat = [1,0,0,0]
        target_left = np.concatenate([xyz_target_left, quat])
        target_right = np.concatenate([xyz_target_right, quat])
        
        
        #Build memory using a set of predictors
        for func in predictor:
            #Predict the warmstart
            x_cur = np.concatenate([xyz_target_left,xyz_target_right])[None,:]
            if isinstance(func, Straight_Regressor):
                traj,_ = func.predict(init_joint0, target_joint)     
            else:
                traj,_ = func.predict(x_cur)
            traj = traj.reshape(-1,dof)
            traj[0,:] = init_joint0
            
            #Plan
            robot.SetActiveDOFValues(init_joint0)
            request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='pose')
            request_traj = add_constraint(request_traj, 'pose', 'l_gripper_tool_frame', target_left,n_steps-1)
            request_traj = add_constraint(request_traj, 'pose', 'r_gripper_tool_frame', target_right,n_steps-1)
            request_traj = set_init(request_traj, 'given_traj', traj)
            duration, result = run_opt(request_traj, env)    

            #Check traj result
            traj = result.GetTraj()
            if check_traj_pose(env,result, target_left, target_right):
                print 'Planning is successfull!'
                x_init.append(x_cur)
                y_init.append(traj.flatten())
                comp_times.append(duration)
                costs.append(result.GetCosts()[0][1])
                break
            else:
                print('Fail to find good solution!')
                continue   

        num_plan += 1
        
        
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [7]:
to_build_database = False
to_augment_database = False
num_traj = 1000
n_steps = 30
limits = dict()
limits[0] = [[0.47, 0.8],[-0.5, 0.5],[0.3,1.4]]
straight = Straight_Regressor(dof = dof, n_steps = n_steps)

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps, predictor = [straight])
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)

In [8]:
print x_init.shape

(1500, 6)


### Reduce the data sample

In [9]:
x_init = x_init[0:1000]
y_init = y_init[0:1000]

### Apply Dimensionality Reduction to the data

In [10]:
from sklearn.decomposition import PCA
y_pca = PCA(n_components = 50)
y_pca.fit(y_init)
y_init_reduced = y_pca.transform(y_init)

### Build Function Approximator

In [11]:
dim_input = len(x_init[0])
is_load_regressor = True

straight = Straight_Regressor(dof = dof, n_steps=n_steps)
nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
gpy_pca = GPy_Regressor(dim_input = dim_input, is_transform='PCA')
bgmr = DP_GLM_Regressor()
bgmr_pca = DP_GLM_Regressor(is_transform='PCA')

if is_load_regressor:
    nn.load_from_file(FILENAME + 'nn')
    gpy.load_from_file(FILENAME + 'gpy')
    gpy_pca.load_from_file(FILENAME + 'gpy_pca')
    bgmr.load_from_file(FILENAME + 'bgmr')
    bgmr_pca.load_from_file(FILENAME + 'bgmr_pca')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=10)
    gpy.save_to_file(FILENAME + 'gpy')
    
    print 'Planning for GPY PCA'
    gpy_pca.fit(x_init, y_init_reduced)
    gpy_pca.save_to_file(FILENAME + 'gpy_pca')

    print 'Planning for bgmr'
    bgmr.fit(x_init,y_init)
    bgmr.save_to_file(FILENAME + 'bgmr')

    print 'Planning for bgmr PCA'
    bgmr_pca.fit(x_init,y_init_reduced)
    bgmr_pca.save_to_file(FILENAME + 'bgmr_pca')

#### Test Function Approximators


### Creating the test cases

In [12]:
create_test = False
if create_test:
    x_test = []
    target_joints = []
    num_test = 500
    for i in range(num_test):
        robot.SetActiveDOFValues(init_joint0)
        target_joint_right,xyz_target_right,target_joint_left,xyz_target_left,target_joint = get_random_pose_both(env,limits,right_arm,left_arm)
        robot.SetActiveDOFValues(target_joint)
        
        quat = [1,0,0,0]
        target_left = np.concatenate([xyz_target_left, quat])
        target_right = np.concatenate([xyz_target_right, quat])
 
        cur_x = (np.concatenate([xyz_target_left, xyz_target_right]))
        x_test.append(cur_x)
        target_joints.append(target_joint)

    x_test = np.vstack(x_test)
    data_test = dict()
    data_test['x_test'] = x_test
    data_test['target_joints'] = target_joints
    test_file = open(FILENAME + 'data_test.npy', 'wb')
    pickle.dump( data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.npy', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    target_joints = data_test['target_joints']
    num_test = len(x_test)
    

In [13]:
def get_ik_both_arm(init_joint, x_cur, num_ik = 5):
    xyz_left = x_cur[0,:3].tolist()
    xyz_right = x_cur[0,3:].tolist()
    quat = [1,0,0,0]
    T_left = openravepy.matrixFromPose( np.r_[quat, xyz_left] )
    T_right = openravepy.matrixFromPose( np.r_[quat, xyz_right] )
    is_success = False
    
    #get a valid IK solution starting from a random configuration
    while is_success is False:
        random_joints = get_random_joints(low_limit, up_limit)
        #Find pose for left
        print 'Finding left'
        robot.SetActiveDOFValues(random_joints)
        robot.SetDOFValues(init_joint[:7], right_arm.GetArmIndices())

        joint_angle_left = ku.ik_for_link(T_left, left_arm, 'l_gripper_tool_frame',
          filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
        if joint_angle_left is None:
            print 'None'
            continue

        #check for collision
        robot.SetDOFValues(joint_angle_left, left_arm.GetArmIndices())
        if env.CheckCollision(robot) or robot.CheckSelfCollision(): continue
            
        #Find pose for right
        print 'Finding right'
        robot.SetActiveDOFValues(random_joints)
        robot.SetDOFValues(init_joint[7:], left_arm.GetArmIndices())
        joint_angle_right = ku.ik_for_link(T_right, right_arm, 'r_gripper_tool_frame',
          filter_options = openravepy.IkFilterOptions.CheckEnvCollisions)
        if joint_angle_right is None:continue

        #check for collision
        robot.SetDOFValues(joint_angle_right, right_arm.GetArmIndices())
        if env.CheckCollision(robot) or robot.CheckSelfCollision(): 
            print 'In collision'
            continue
            
        is_success = True
    
    
    #given the successful random_joints, get all valid IK solutions and shuffle them
    robot.SetActiveDOFValues(random_joints)
    robot.SetDOFValues(init_joint[:7], right_arm.GetArmIndices())

    joint_angle_lefts = ku.ik_for_link(T_left, left_arm, 'l_gripper_tool_frame',
      filter_options = openravepy.IkFilterOptions.CheckEnvCollisions, return_all_solns = True)
    
    robot.SetActiveDOFValues(random_joints)
    robot.SetDOFValues(init_joint[7:], left_arm.GetArmIndices())
    joint_angle_rights = ku.ik_for_link(T_right, right_arm, 'r_gripper_tool_frame',
          filter_options = openravepy.IkFilterOptions.CheckEnvCollisions, return_all_solns = True)    
    
    joint_angles = [np.concatenate([joint_angle_right,joint_angle_left])]
    
    j = 0
    while len(joint_angles) < num_ik and j < 30:
        if len(joint_angle_lefts) == 0 or len(joint_angle_rights) == 0:
            break
        np.random.shuffle(joint_angle_lefts)
        np.random.shuffle(joint_angle_rights)
        joint_angle_left = joint_angle_lefts[0]
        joint_angle_right = joint_angle_rights[0]
        joint_angle = np.concatenate([joint_angle_right,joint_angle_left])
        robot.SetActiveDOFValues(joint_angle)
        j += 1
        if env.CheckCollision(robot) or robot.CheckSelfCollision(): 
            continue
        else:
            joint_angles += [joint_angle]
            
    return joint_angles

In [14]:
def compute_costs(traj):
    cost = 0
    for i in range(len(traj)-1):
        cost += np.linalg.norm(traj[i]-traj[i+1])
    return cost

In [15]:
#load gpy for joint angle
FILENAME_AUX = 'data/dual_arm_fixed_init/'
gpy_aux = GPy_Regressor(dim_input = 14)
gpy_aux.load_from_file(FILENAME_AUX + 'gpy')

In [16]:
#load gpy and nn for cost
FILENAME_AUX = 'data/dual_arm_fixed_init/'
gpy_cost = GPy_Regressor(dim_input = 14)
gpy_cost.load_from_file(FILENAME_AUX + 'gpy_cost')

nn_cost = NN_Regressor()
nn_cost.load_from_file(FILENAME_AUX + 'nn_cost')

### Setting up Parallel Programming for Ensemble Method

In [ ]:
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint0
        robot.SetActiveDOFValues(init_joint0)
        request_traj = set_init(request_traj, 'given_traj', traj)
        request_trajs += [request_traj]
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj_pose(env,result[method_name], target_left, target_right)


In [70]:
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import Lock

mutex = Lock()
   
def run_opt_pose_special(request):
    robot.SetActiveDOFValues(init_joint0)
    s = json.dumps(request) 
    prob = trajoptpy.ConstructProblem(s, env)
    tic = time.time()
    try:
        result = trajoptpy.OptimizeProblem(prob) # do optimization
    except:
        print 'Fault in optimization'
        return False, None, None, None
    toc = time.time()
    print("Optimization is completed in {} s!".format(toc-tic))
    duration = toc-tic
    
    #check the result
    try:
        success = check_traj_pose(env,result, target_left, target_right)    
        result_new = dict()
        result_new['traj'] = result.GetTraj()
        result_new['costs'] = result.GetCosts()[0][1]
        cost = result.GetCosts()[0][1]
        result_new['status'] = result.GetStatus()
        result_new['func_evals'] = result.GetNumFuncEval()[0]
        result_new['qp_solves'] = result.GetNumQPSolve()[0]
        return success, duration, cost, result_new
    except:
        'Got an exception when checking the trajectory'
        return False, None, None, None
    
    
def run_opt_special(request, target_joint):
    robot.SetActiveDOFValues(init_joint0)
    s = json.dumps(request) 
    prob = trajoptpy.ConstructProblem(s, env)
    tic = time.time()
    try:
        result = trajoptpy.OptimizeProblem(prob) # do optimization
    except:
        print 'Fault in optimization'
        return False, None, None, None
    toc = time.time()
    print("Optimization is completed in {} s!".format(toc-tic))
    duration = toc-tic
    
    #check the result
    try:
        success = check_traj(env,result, target_joint)    
        result_new = dict()
        result_new['traj'] = result.GetTraj()
        result_new['costs'] = result.GetCosts()[0][1]
        cost = result.GetCosts()[0][1]
        result_new['status'] = result.GetStatus()
        result_new['func_evals'] = result.GetNumFuncEval()[0]
        result_new['qp_solves'] = result.GetNumQPSolve()[0]
        return success, duration, cost, result_new
    except:
        'Got an exception when checking the trajectory'
        return False, None, None, None

    
def function_callback(result):
    with mutex:
        print 'Obtain result:'
        print result[0]
        global count_proc
        global num_proc
        count_proc+= 1
        global ensemble_result
        ensemble_result = result
        print count_proc, num_proc
        if result[0]:
            print 'Terminating the mp'
            pool.terminate()
            return
        if count_proc == num_proc:
            print 'All fails, terminating the mp'
            pool.terminate()

In [72]:
method_names = ['STD    ',  'NN     ', 'GPY    ', 'GPY_PCA',  'BGMR','BGMR_PCA', 'METRIC GPY_PCA','METRIC DIRECT', 'ensemble']
methods = [straight, nn, gpy,gpy_pca, bgmr, bgmr_pca]#
results = dict()
result,success, duration = dict(),dict(),dict()
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    
num_test = 250
ik_times = []
quat = [1,0,0,0]
ensemble_result = None
for it in range(num_test):
    print('{}th Planning'.format(it))
    
    #setting up the problem case
    #index = np.random.randint(0,len(x_test))
    index = it
    x_cur = x_test[index:index+1,:]
    target_joint = target_joints[index]
    robot.SetActiveDOFValues(init_joint0)
    
    #Other warmstart methods
    target_left = np.concatenate([x_cur[0,0:3], quat])
    target_right = np.concatenate([x_cur[0,3:], quat])
    
    
    request_trajs = []
    for i,method in enumerate(methods):
        method_name = method_names[i]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict(init_joint0, target_joint)
        elif isinstance(method, DP_GLM_Regressor):
            traj,cov = method.predict(x_cur, return_gmm = True)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = y_pca.inverse_transform(traj)
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint0
        robot.SetActiveDOFValues(init_joint0)
        request_traj = define_request(coeffs = coeffs,init_type='given_traj',constraint_type='pose')
        request_traj = add_constraint(request_traj, 'pose', 'l_gripper_tool_frame', target_left,n_steps-1)
        request_traj = add_constraint(request_traj, 'pose', 'r_gripper_tool_frame', target_right,n_steps-1)

        request_traj = set_init(request_traj, 'given_traj', traj)
        request_trajs += [request_traj]
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj_pose(env,result[method_name], target_left, target_right)

    
    #run GPY_aux with joint planning
    method_name = method_names[-3]
    num_ik = 5
    trajs = []
    targets = []
    costs = []
    tic = time.time()
    robot.SetActiveDOFValues(target_joint)
    joint_angles = get_ik_both_arm(init_joint0,x_cur, num_ik)
    joint_angles.append(target_joint)
    for j in range(len(joint_angles)):
        joint_angle = joint_angles[j]
        traj,cov = gpy_aux.predict(joint_angle[None,:])
        traj = traj.reshape(-1,dof)
        trajs.append(traj)
        targets.append(joint_angle)
        costs.append(compute_costs(traj))
    toc = time.time()
    print toc-tic
    ik_times.append(toc-tic)
    min_index = np.argmin(costs)
    print('Min index based on init_guess: {}'.format(min_index))
    print costs
    
    traj = trajs[min_index]
    target_chosen = targets[min_index]
    metric_target = target_chosen.copy()
    request_traj_aux = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj_aux = add_constraint(request_traj_aux, 'joint', '', target_chosen,-1)
    
    traj[0,:] = init_joint0
    robot.SetActiveDOFValues(init_joint0)
    request_traj_aux = set_init(request_traj_aux, 'given_traj', traj)
    request_trajs += [request_traj_aux]
    duration[method_name], result[method_name] = run_opt(request_traj_aux, env)    
    success[method_name] = check_traj(env,result[method_name], target_chosen)

    #run GPy_cost with joint planning
    method_name = method_names[-2]
    num_ik = 5
    trajs = []
    targets = []
    costs = []
    for j in range(len(joint_angles)):
        joint_angle = joint_angles[j]
        cost,_ = gpy_cost.predict(joint_angle[None,:])
        costs.append(cost)
    min_index = np.argmin(costs)
    print('Min index based on cost: {}'.format(min_index))
    print costs
    joint_angle = joint_angles[min_index]
    traj,cov = gpy_aux.predict(joint_angle[None,:])
    traj = traj.reshape(-1,dof)
    target_chosen = joint_angle
    request_traj_aux = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj_aux = add_constraint(request_traj_aux, 'joint', '', target_chosen,-1)
    
    traj[0,:] = init_joint0
    robot.SetActiveDOFValues(init_joint0)
    request_traj_aux = set_init(request_traj_aux, 'given_traj', traj)
    request_trajs += [request_traj_aux]
    duration[method_name], result[method_name] = run_opt(request_traj_aux, env)    
    success[method_name] = check_traj(env,result[method_name], target_chosen)
    
    #ensemble method
    print("Start ensemble")
    num_proc = len(request_trajs)
    pool = mp.Pool(processes = num_proc)
    count_proc = 0
    ticc = time.time()
    for request in request_trajs:
        if i < num_proc-2:
            pool.apply_async(run_opt_pose_special, args = (request, ), callback = function_callback)
        else:
            pool.apply_async(run_opt_special, args = (request,metric_target ), callback = function_callback)
            
    time.sleep(10)
    pool.terminate()
    pool.close()
    pool.join()
    tocc = time.time()
    print 'Ensemble takes ' + str(tocc-ticc)
    results['ensemble']['costs'].append(ensemble_result[2])
    results['ensemble']['successes'].append(ensemble_result[0])      
    results['ensemble']['comp_times'].append(tocc-ticc)
    results['ensemble']['func_evals'].append(ensemble_result[3]['func_evals'])
    results['ensemble']['qp_solves'].append(ensemble_result[3]['qp_solves'])

    #Record the result
    for method_name in method_names[:-1]:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])
    
    print_result(results, method_names)

0th Planning
Optimization is completed in 0.304868936539 s!
Optimization is completed in 0.469851016998 s!
Optimization is completed in 0.745054006577 s!
Optimization is completed in 0.98236989975 s!
Optimization is completed in 0.589696884155 s!
Optimization is completed in 0.682738780975 s!
Finding left
Finding right
0.0906209945679
Min index based on init_guess: 5
[8.5835845414944156, 7.3830530428308476, 7.2550186266445937, 8.5882500482479429, 7.2609883902219101, 7.2492712519353795]
Optimization is completed in 0.31368303299 s!
Min index based on cost: 5
[array([[ 2.4899]]), array([[ 1.8778]]), array([[ 1.7948]]), array([[ 2.4928]]), array([[ 1.8134]]), array([[ 1.7943]])]
Optimization is completed in 0.313643932343 s!
Start ensemble
Optimization is completed in 0.425013065338 s!
Obtain result:
True
1 8
Terminating the mp
Ensemble takes 0.637489080429
 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD     	& 100.000 	& 0.30$\pm$0.00 	& 1.74$\pm$0.00 

In [69]:
print("Start ensemble")
num_proc = len(request_trajs)
pool = mp.Pool(processes = num_proc)
count_proc = 0
ticc = time.time()
c = 0
pool.apply_async(run_opt_special, args = (request_trajs[-1],metric_targets[-1] ), callback = function_callback)


Start ensemble


Optimization is completed in 2.30768108368 s!
Target joint is not reached!
Obtain result:
False
1 4


### Printing Results

### Save Result

In [73]:
pickle.dump(results,open(FILENAME + 'result_new2.pkl', 'wb') )

### Load Result 

In [ ]:
results = pickle.load(open(FILENAME + 'result_new2.pkl', 'rb'))

### Printing Results Tied to Standard Success

In [48]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    
    print('{0}: \t {1:.3f} \t {2:.2f} \t {3:.3f} \t {4:.3f} \t {5:.3f}'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD    : 	 1.000 	 1.17 	 1.849 	 9.213 	 8.280
NN     : 	 0.762 	 1.15 	 1.841 	 12.088 	 10.864
GPY    : 	 0.707 	 1.84 	 1.811 	 13.026 	 11.707
GPY_PCA: 	 0.756 	 1.66 	 1.803 	 12.185 	 10.952
BGMR: 	 0.750 	 1.38 	 1.807 	 12.285 	 11.041
BGMR_PCA: 	 0.793 	 1.33 	 1.860 	 11.623 	 10.446
METRIC GPY_PCA: 	 0.927 	 0.63 	 1.426 	 9.941 	 8.934
METRIC DIRECT: 	 0.933 	 0.64 	 1.423 	 9.876 	 8.876
ensemble: 	 1.000 	 1.24 	 1.532 	 9.213 	 8.280
